#RC4

In [8]:
import time
import random
import hashlib
import os
import binascii
import string

from cryptography.hazmat.primitives import asymmetric, hashes, kdf, ciphers, padding
from cryptography.hazmat.backends import default_backend

print("Librerías necesarias importadas.")

def generate_random_text_file(num_words, filename):
    def _generate_word():
        length = random.randint(3, 10)
        return ''.join(random.choice(string.ascii_lowercase) for _ in range(length))

    words = [_generate_word() for _ in range(num_words)]
    with open(filename, 'w') as f:
        f.write(' '.join(words))
    print(f"Se generaron {num_words} palabras aleatorias y se guardaron en '{filename}'.")

print("Función auxiliar 'generate_random_text_file' definida.")

Librerías necesarias importadas.
Función auxiliar 'generate_random_text_file' definida.


In [9]:
word_counts = [10, 100, 1000, 10000, 100000, 1000000, 10000000]
file_paths = {}

print("Verificando y generando archivos de texto para todos los tamaños...")
for num_words in word_counts:
    filename = f"input_shared_{num_words}.txt"
    file_paths[num_words] = filename

    if not os.path.exists(filename):
        generate_random_text_file(num_words, filename)
    else:
        print(f"El archivo '{filename}' ya existe. Se utilizará el archivo existente.")

print("Archivos de texto listos para usar.")

Verificando y generando archivos de texto para todos los tamaños...
El archivo 'input_shared_10.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_100.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_1000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_10000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_100000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_1000000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_10000000.txt' ya existe. Se utilizará el archivo existente.
Archivos de texto listos para usar.


In [10]:
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import ARC4
from cryptography.hazmat.backends import default_backend

rc4_results = []

print("Iniciando la medición de rendimiento de RC4...")

for num_words in word_counts:
    filename = file_paths[num_words]

    start_time = time.time()
    with open(filename, 'r') as f:
        plaintext_str = f.read()
    plaintext = plaintext_str.encode('utf-8')
    T_E1 = time.time() - start_time

    start_time = time.time()
    key = os.urandom(16)
    print(f"  Clave de cifrado/descifrado (RC4): {key.hex()}")
    T_E2 = time.time() - start_time

    start_time = time.time()
    cipher = Cipher(ARC4(key), mode=None, backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    T_E3 = time.time() - start_time

    start_time = time.time()
    decryptor = cipher.decryptor()
    decrypted_text_bytes = decryptor.update(ciphertext) + decryptor.finalize()
    T_E4 = time.time() - start_time

    T_Total = T_E1 + T_E2 + T_E3 + T_E4

    input_chars = len(plaintext_str)
    output_chars = len(ciphertext)

    rc4_results.append({
        "algoritmo": "RC4",
        "#palabras": num_words,
        "#caracteres_entrada": input_chars,
        "#caracteres_salida": output_chars,
        "T-E1": T_E1,
        "T-E2": T_E2,
        "T-E3": T_E3,
        "T-E4": T_E4,
        "T_Total": T_Total
    })

    print(f"Procesamiento completado para {num_words} palabras.")

print("Resultados de rendimiento de RC4:")
df_rc4_results = pd.DataFrame(rc4_results)
display(df_rc4_results)

print("Medición de rendimiento de RC4 completada.")

Iniciando la medición de rendimiento de RC4...
  Clave de cifrado/descifrado (RC4): 78b3e56ce8568f768275d7c5680ba5fa
Procesamiento completado para 10 palabras.
  Clave de cifrado/descifrado (RC4): cf8d494e8ecce8b02c4bf49102ab5a4b
Procesamiento completado para 100 palabras.
  Clave de cifrado/descifrado (RC4): c1e2c3abc91da79c7d20c22de1a4f2ef
Procesamiento completado para 1000 palabras.
  Clave de cifrado/descifrado (RC4): 1fb9ee1fe01025521b6ad83bed376284
Procesamiento completado para 10000 palabras.
  Clave de cifrado/descifrado (RC4): a71009b7458c97f5812991798b6919ae
Procesamiento completado para 100000 palabras.


/tmp/ipython-input-228615182.py:3: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import ARC4


  Clave de cifrado/descifrado (RC4): 55b9f3774e63bf774ae988400a8737fc
Procesamiento completado para 1000000 palabras.
  Clave de cifrado/descifrado (RC4): c3e8639ad4b3930f64a4d44eec2ece1d
Procesamiento completado para 10000000 palabras.
Resultados de rendimiento de RC4:


,algoritmo,#palabras,#caracteres_entrada,#caracteres_salida,T-E1,T-E2,T-E3,T-E4,T_Total
0,RC4,10,70,70,0.049385,0.000083,0.034343,0.031545,0.115356
1,RC4,100,754,754,0.000200,0.000032,0.000137,0.000039,0.000408
2,RC4,1000,7512,7512,0.000090,0.000016,0.000062,0.000033,0.000201
3,RC4,10000,74568,74568,0.000113,0.000017,0.000248,0.010337,0.010715
4,RC4,100000,748950,748950,0.012511,0.000116,0.010258,0.003710,0.026595
5,RC4,1000000,7499596,7499596,0.046742,0.000183,0.037280,0.054676,0.138881
6,RC4,10000000,74997531,74997531,0.543738,0.002224,0.926984,0.945078,2.418024


Medición de rendimiento de RC4 completada.
